In [1]:
%env CUDA_VISIBLE_DEVICES=4

env: CUDA_VISIBLE_DEVICES=4


In [2]:
from magma.magma import Magma 
import torch 
import seaborn as sns
import matplotlib.pyplot as plt
import os 
from activations.torch import Rational
from magma.adapters import Adapter
import numpy as np
from magma.utils import get_tokenizer
from magma.datasets import ImgCptDataset
from magma.transforms import get_transforms
from magma.config import MultimodalConfig
config = MultimodalConfig.from_yml(
            './../fb20-dgx2-configs/dev.yml'
        )
sns.set_style("darkgrid")
sns.color_palette('husl', 9)
from copy import deepcopy


/home/ml-mmeuer/anaconda3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_7250 = Magma.from_checkpoint(checkpoint_path='./../model_checkpoints/switch2_7250.pt', config_path='./../fb20-dgx2-configs/dev.yml').cuda()
dataset = ImgCptDataset(config.train_dataset_dir, model_7250.tokenizer, model_7250.transforms)

Loading GPTJ language model...
From EleutherAI/gpt-j-6B
50256
loading magma checkpoint from: ./../model_checkpoints/switch2_7250.pt
magma successfully loaded


In [ ]:
for n,m in model_7250.named_parameters(): 
    print(n)

lm.transformer.wte.weight
lm.transformer.h.0.ln_1.weight
lm.transformer.h.0.ln_1.bias
lm.transformer.h.0.attn.k_proj.weight
lm.transformer.h.0.attn.v_proj.weight
lm.transformer.h.0.attn.q_proj.weight
lm.transformer.h.0.attn.out_proj.weight
lm.transformer.h.0.mlp.0.fc_in.weight
lm.transformer.h.0.mlp.0.fc_in.bias
lm.transformer.h.0.mlp.0.fc_out.weight
lm.transformer.h.0.mlp.0.fc_out.bias
lm.transformer.h.0.mlp.1.switch_logits
lm.transformer.h.0.mlp.1.adapter.0.weight
lm.transformer.h.0.mlp.1.adapter.0.bias
lm.transformer.h.0.mlp.1.adapter.1.f.numerator
lm.transformer.h.0.mlp.1.adapter.1.f.denominator
lm.transformer.h.0.mlp.1.adapter.2.weight
lm.transformer.h.0.mlp.1.adapter.2.bias
lm.transformer.h.1.ln_1.weight
lm.transformer.h.1.ln_1.bias
lm.transformer.h.1.attn.k_proj.weight
lm.transformer.h.1.attn.v_proj.weight
lm.transformer.h.1.attn.q_proj.weight
lm.transformer.h.1.attn.out_proj.weight
lm.transformer.h.1.mlp.0.fc_in.weight
lm.transformer.h.1.mlp.0.fc_in.bias
lm.transformer.h.1.mlp.

In [5]:
for n, m in model_7250.named_parameters():
    print(n, m.requires_grad)

lm.transformer.wte.weight False
lm.transformer.h.0.ln_1.weight False
lm.transformer.h.0.ln_1.bias False
lm.transformer.h.0.attn.k_proj.weight False
lm.transformer.h.0.attn.v_proj.weight False
lm.transformer.h.0.attn.q_proj.weight False
lm.transformer.h.0.attn.out_proj.weight False
lm.transformer.h.0.mlp.0.fc_in.weight False
lm.transformer.h.0.mlp.0.fc_in.bias False
lm.transformer.h.0.mlp.0.fc_out.weight False
lm.transformer.h.0.mlp.0.fc_out.bias False
lm.transformer.h.0.mlp.1.switch_logits True
lm.transformer.h.0.mlp.1.adapter.0.weight True
lm.transformer.h.0.mlp.1.adapter.0.bias True
lm.transformer.h.0.mlp.1.adapter.1.f.numerator True
lm.transformer.h.0.mlp.1.adapter.1.f.denominator True
lm.transformer.h.0.mlp.1.adapter.2.weight True
lm.transformer.h.0.mlp.1.adapter.2.bias True
lm.transformer.h.1.ln_1.weight False
lm.transformer.h.1.ln_1.bias False
lm.transformer.h.1.attn.k_proj.weight False
lm.transformer.h.1.attn.v_proj.weight False
lm.transformer.h.1.attn.q_proj.weight False
lm.tra

In [ ]:
n_adapter = len([m for m in model_7250.modules() if isinstance(m, Adapter)])

variations = [[0,n_adapter-1], [0], [n_adapter-1], [0,1,2], [n_adapter-1,n_adapter-2,n_adapter-3], [0, n_adapter//2, n_adapter], [None]]

for var in variations:
    
    model = deepcopy(model_7250)
    j = 0
    for i, ad in enumerate(model.modules()): 
        
        if isinstance(ad, Adapter):
            
            if j in var:
                ad.fixed_idx = 0
            else:
                ad.fixed_idx = 1
            ad.training = False
            j += 1
    n_samples = 10
    average_loss = []
    for i in range(n_samples):
        img, caption = dataset[i]
        img = img.half().cuda()
        caption = caption.cuda()
        with torch.no_grad():
            output = model(img, caption)
        average_loss.append(output['loss'].item())
        torch.cuda.empty_cache()
        
    print(f'Average loss with Adapters at {var}: {np.mean(average_loss)}')
        

FIXED IDX 0
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 0
FIXED IDX 0
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 0
FIXED IDX 0
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXED IDX 1
FIXE

In [ ]:
print(f'Average loss: {np.mean(average_loss)}')

Average loss: 1.85009765625
